# Wave Interference

The plotted function is the superimposition of $n$ 2d-sine  functions centered in $(0,Y_i)$:
$$f(x,y)=\sin\big(p\cdot\sqrt{x^2+(y-Y_i)^2}\big)$$

The parameters that can be changed are:

-- $n$ the number of sources (start with 2)

-- $d$ the distance between sources

-- $p$ the periodicity. Higher periodicity means shorter wavelength. $p =2\pi / \lambda$. Note that in electron microscopy the usual convention is to define the wave vector as $k=1/\lambda$. 

-- $f$ the field of view.

One sees the directions of space where intereferences are constructive / destructive, with the angle between the outgoing beams increasing with decreasing $p$, and decreasing source spacing.

## Hints for the use

Start with $n=2$. 

Press on "run interact" to initiate the plotting after changing the parameters. 

With many sources, the intereference patterns close to the source is quite intricated. One needs to go to the far field (increase $f$ the field of view) to see the formation of the outgoing beams.

With more sources (try up to 10 sources), the outgoing beams are getting thinner.

In [5]:
import matplotlib.pyplot as plt
import numpy as np
import matplotlib

from ipywidgets import interact, interactive, fixed, interact_manual
import ipywidgets as widgets
matplotlib.rcParams['figure.figsize'] = (16,16)


In [6]:
# n is the number of sources
# d distance between sources
# p changes periodicity
# f field of view

def waveplot2d(n,d,p,f):
    x=np.linspace(0,f,2001)
    y=np.linspace(-f/2,f/2,2001)
    X, Y = np.meshgrid(x, y) 
    c=np.linspace(-(n-1)*d/2,(n-1)*d/2,n)
    c0=np.linspace(0,0,n)
    print(c)
    Z=0*X+0*Y
    for i in c:
        Zi=(np.sin(p*np.sqrt(X**2+(Y-i)**2)))**1
        Z=Z+Zi
    plt.figure()
    cp=plt.contourf(X,Y,Z)
    plt.plot(c0,c,'.',markersize=20,color='blue')
    plt.axes().set_aspect('equal')
    plt.colorbar(cp)
    plt.set_cmap('coolwarm') 
    plt.show()
    
#waveplot2d(2,10,3)  

interact_manual(waveplot2d,n=(1,10,1),d=(2,20,1),p=(1,5,1),f=(50,900,50))




interactive(children=(IntSlider(value=5, description='n', max=10, min=1), IntSlider(value=11, description='d',…

<function __main__.waveplot2d(n, d, p, f)>